# Assignment 2

## Roll number: 


### Instructions
 * Fill in the roll-number in the cell above.
 * Code must be submitted in Python in jupyter notebooks. We highly recommend using anaconda/miniconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 *  Most of the questions require you to **code your own functions** unless there is a need to call in the abilities of the mentioned libraries, such as Visualisation from Open3D. Make sure your code is modular since you will be reusing them for future assignments. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 *  All the representations are expected to be in a right-hand coordinate system.
<!--  * Answer to the descriptive questions should be answered in your own words. Copy-paste answers will lead to penalty. -->
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.
 * Commit the notebooks in the repo and any other results files under the result folder in the GitHub Classroom repo. 
 * This is a group assignment. Discussions are encouraged but any sharing of code among different teams will be penalized. 

## SECTION 1: ICP with SVD

### 1.1 Perform Procrustes alignmenton two point clouds with (given) known correspondences. (5 Points)

Let X be your point cloud observed from the initial pose P1. You then transform it to a new pose P2. Now you wish to apply ICP to recover transformation between (X & P1) and (X & P2).

Use toothless.ply point cloud and perform the alignment between the two point clouds using procrustes alignment. Your task is to write a function that takes two point clouds as input wherein the corresponding points between the two point clouds are located at the same index and returns the transformation matrix between them. Compute the alignment error after aligning the two point clouds.

<b>Use root mean squared error (RSME) as the alignment error metric.</b>

Make sure your code is modular as we will use this function in the next sub-part.

We will again use our own getTransform function to generate the 4x4 Transformation matrix to transform the pointcloud to P2, so make sure your code works for any general Transformation matrix

In [2]:
import numpy as np
import math

def alpha_rot(alpha):
    return np.array([
        [   1.0,    0.0,                0.0             ],
        [   0.0,    math.cos(alpha),   -math.sin(alpha) ],
        [   0.0,    math.sin(alpha),    math.cos(alpha) ]
        ],dtype=np.float32)

def beta_rot(beta):
    return np.array([
        [   math.cos(beta),     0.0,    math.sin(beta)  ],
        [   0.0,                1.0,    0.0             ],
        [  -math.sin(beta),     0.0,    math.cos(beta)  ]
        ],dtype=np.float32)

def gamma_rot(gamma):
    return np.array([
        [   math.cos(gamma),   -math.sin(gamma),    0.0 ],
        [   math.sin(gamma),    math.cos(gamma),    0.0 ],
        [   0.0,                0.0,                1.0 ]
        ],dtype=np.float32)
        
def euler2rm(alpha, beta, gamma):
    return np.matmul(alpha_rot(alpha),np.matmul(beta_rot(beta),gamma_rot(gamma)))

def getTransform():
    # Random rotation matrices for X, Y, and Z axes
    R = euler2rm(np.random.uniform(0, 2 * np.pi),
                 np.random.uniform(0, 2 * np.pi),
                 np.random.uniform(0, 2 * np.pi))
    
    # Random translation matrix
    T = np.array([
        [np.random.uniform(-500, 500)],
        [np.random.uniform(-500, 500)],
        [np.random.uniform(-500, 500)]
    ])
    
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = R
    transformation_matrix[:3, 3] = T.flatten()
    
    return transformation_matrix

In [3]:
T = getTransform()
def icp_known_correspondences(T):
    #### YOUR CODE HERE ####
    pass

### 1.2 Implement ICP algorithm with unknown correspondences. (5 Points)

Your task is to write a function that implements ICP and takes two point clouds as input wherein the correspondances are unknown. Visualize the pointclouds and plot their individual coordinate frames as you perform ICP over them. Compute the alignment error in each iteration. 

Refer to Shubodh's notes to compute correspondences: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=a25686ce1a11409d838d47bcac43ab4b&pm=s#bb9aaf2e316b4db3b399df1742f0444c


In [11]:
import numpy as np
import open3d as o3d
from sklearn.neighbors import NearestNeighbors

def invert_transform(matrix):
    rotation_matrix = matrix[:3, :3]
    translation_vector = matrix[:3, 3]

    inv_rotation_matrix = np.transpose(rotation_matrix)
    inv_translation_vector = -np.matmul(inv_rotation_matrix, translation_vector)

    inv_matrix = np.eye(4)
    inv_matrix[:3, :3] = inv_rotation_matrix
    inv_matrix[:3, 3] = inv_translation_vector
    
    return inv_matrix

def transform_point_cloud(point_cloud, transformation_matrix, downsample=1.0):
    if downsample < 1.0:
        indices = np.arange(0, point_cloud.shape[0], int(1.0 / downsample))
        point_cloud = point_cloud[indices]

    rotated_points = np.dot(point_cloud, transformation_matrix[:3, :3].T)
    transformed_points = rotated_points + transformation_matrix[:3, 3]

    return transformed_points

def compute_correspondences(pcl0,pcl1):
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(pcl0)
    distances, indices = nbrs.kneighbors(pcl1)
    matched_pcl0 = pcl1[indices.ravel()]

    return matched_pcl0, distances.ravel()

def icp_with_svd(pcl0, pcl1, distances):
    # Compute centroids
    centroid_pcl0 = np.mean(pcl0, axis=0)
    centroid_pcl1 = np.mean(pcl1, axis=0)

    # Center the point clouds
    centered_pcl0 = pcl0 - centroid_pcl0
    centered_pcl1 = pcl1 - centroid_pcl1

    # Compute cross covariance matrix
    H = centered_pcl0.T @ centered_pcl1

    # Compute SVD
    U, _, Vt = np.linalg.svd(H)
    R = U @ Vt

    # Handle reflection case
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = U @ Vt

    # Compute translation
    t = centroid_pcl0 - R @ centroid_pcl1

    # Create transformation matrix
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t

    # Check for convergence
    mean_residual = np.mean(distances)

    return T, mean_residual

downsample = 0.01
pcl = o3d.io.read_point_cloud('data/toothless.ply')
pcl_array = transform_point_cloud(np.array(pcl.points), np.eye(4), downsample)

transform0 = getTransform()
transform1 = getTransform()

pcl0 = transform_point_cloud(pcl_array, transform0)
pcl1 = transform_point_cloud(pcl_array + np.random.normal(0.0, 0, pcl_array.shape), transform1)

pcl0_geom = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcl0))
frame0 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[0, 0, 0])
pcl0_geom.paint_uniform_color([1.0, 0.0, 0.0])
pcl1_geom = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcl1))
frame1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100.0, origin=[0, 0, 0])
pcl1_geom.paint_uniform_color([0.0, 0.0, 1.0])

vis = o3d.visualization.Visualizer()
vis.create_window()

vis.add_geometry(pcl0_geom)
vis.add_geometry(frame0)
vis.add_geometry(pcl1_geom)
vis.add_geometry(frame1)

vis.poll_events()
vis.update_renderer()

error = np.inf 
opti_transform = np.eye(4)

print("Trasform 0:")
print(np.array_str(transform0, precision=2, suppress_small=True))
print("Trasform 1:")
print(np.array_str(transform1, precision=2, suppress_small=True))

for i in range(400):
    corresponded_pcl0, distances = compute_correspondences(pcl0,pcl1)
    svd_transform, error = icp_with_svd(corresponded_pcl0, pcl1, distances)
    pcl1 = transform_point_cloud(pcl_array, svd_transform@transform1)
    pcl1_geom.points = o3d.utility.Vector3dVector(pcl1)
    frame1.vertices = o3d.utility.Vector3dVector(transform_point_cloud(np.array(frame0.vertices),svd_transform))
    vis.update_geometry(pcl1_geom)
    vis.update_geometry(frame1)
    vis.poll_events()
    vis.update_renderer()
    if error < 9e-2:
        pcl0_geom.paint_uniform_color([0.0, 1.0, 0.0])
        pcl1_geom.paint_uniform_color([0.0, 1.0, 0.0])
        vis.update_geometry(pcl0_geom)
        vis.update_geometry(pcl1_geom)
        vis.update_geometry(frame1)
        vis.poll_events()
        vis.update_renderer()
        print("Converged")
        print("Expected transformation matrix:")
        print(np.array_str(transform0 @ invert_transform(transform1), precision=2, suppress_small=True))
        print("Final transformation matrix:")
        print(np.array_str(svd_transform, precision=2, suppress_small=True))
        break

if error > 9e-2:
    print("Failed to converge")
vis.run()
vis.destroy_window()

Trasform 0:
[[  -0.19    0.31    0.93 -202.37]
 [   0.1     0.95   -0.29 -445.37]
 [  -0.98    0.04   -0.21  417.39]
 [   0.      0.      0.      1.  ]]
Trasform 1:
[[  -0.27   -0.33    0.9  -472.05]
 [  -0.65   -0.63   -0.43  103.7 ]
 [   0.71   -0.7    -0.05   68.43]
 [   0.      0.      0.      1.  ]]


KeyboardInterrupt: 

## SECTION 2: ICP with Lie Groups

### 2.1 Predict the Transformation matrix between 2 point clouds with known correspondences (15 Points)
Perform the same task as 1.1 using Lie Group Optimization from scratch to predict the transformation between the 2 point clouds.

Refer: https://saishubodh.notion.site/Mobile-Robotics-Navigating-from-Theory-to-Application-0b65a9c20edd4081978f4ffad917febb?p=ee55fe5689794693910ab7861bef067b&pm=s#7b82d84766a84b63b91d859579e4886b


In [4]:
T = getTransform()
def icp_with_lie(T):
    #### YOUR CODE HERE ####
    pass

## SECTION 3: Pose Graph Optimization with G2O
### Objective (5 Points)
A robot is travelling in a oval trajectory. It is equipped with wheel odometry for odometry information and RGBD sensors for loop closure information. Due to noise in wheel odometry it generates a noisy estimate of the trajectory. Our task is to use loop closure pairs to correct the drift.

We pose this problem as a pose graph optimization problem. In our graph, poses are the vertices and constraints are the edges. 

References:

1.) Class notes: https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3

2.) Cyrill Stachniss lecture: https://www.youtube.com/watch?v=uHbRKvD8TWg 

### Given: 
In practical scenarios, we'd obtain the following from our sensors after some post-processing:

1. Initial position
2. Odometry Contraints/Edges: This "edge" information tells us relative transformation between two nodes. These two nodes are consecutive in the case of Odometry but not in the case of Loop Closure (next point).
3. Loop Closure Contraints/Edges: Remember that while optimizing, you have another kind of "anchor" edge as you've seen in 1. solved example.

You have been given a text file named `edges.txt` (in `data/`) which has all the above 3 and it follows G2O's format (as explained in class, [link here](https://saishubodh.notion.site/G2O-Edge-Types-d9f9ff63c77c4ceeb84b1e49085004e3) ). The ground truth is `gt.txt`.

Install g2o as mentioned in `g2o.ipynb` and optimise `edges.txt`

In [5]:
### YOUR CODE HERE ###

### Evo (10 Points)
We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [6]:
### YOUR CODE HERE ###